In [1]:
from __future__ import division
import sys
import cv2
import os
import numpy as np
import argparse
from depth_evaluation_utils import *

In [2]:
class ARGS():
    kitti_dir = '../../TrainingAndValData/Kitti_raw/'
    pred_file = '../../sfmLearner_checkpoints/outputs/depth_noSSIM/model-192572.npy'
    test_file_list = '../data/kitti/test_files_eigen.txt'
    min_depth = 1e-3
    max_depth = 80
args = ARGS()


In [3]:
def get_available_data(dataset_dir, test_files):
    
    def dircheck(dirs, folder):
        for directory in dirs:
            if folder==directory:
                return True
        return False

    dirs = next(os.walk(dataset_dir))[1] # get list of all child dirs
    available_test_files = []
    # for every test_file, check whether raw data is available
    counter = 0
    for i in range(len(test_files)): 
        folder = test_files[i][:10]
        if dircheck(dirs, folder):
            available_test_files.append(test_files[i])
        else:
            counter +=1
    print("No. of test files not available:", counter)
    return available_test_files

In [4]:
# read depth predictions
pred_depths = np.load(args.pred_file)
# read paths
test_files = read_text_lines(args.test_file_list)


In [5]:
test_files = get_available_data(args.kitti_dir, test_files)

# from paths - read ground truth files
gt_files, gt_calib, im_sizes, im_files, cams = read_file_data(test_files, args.kitti_dir)
num_test = len(im_files)

No. of test files not available: 150


In [11]:
velo2cam = read_calib_file(gt_calib[0] + 'calib_velo_to_cam.txt')
np.array(map(float, velo2cam['R'].split()))

['7.533745e-03',
 '-9.999714e-01',
 '-6.166020e-04',
 '1.480249e-02',
 '7.280733e-04',
 '-9.998902e-01',
 '9.998621e-01',
 '7.523790e-03',
 '1.480755e-02']

In [8]:
t_id = 0
camera_id = cams[t_id]  # 2 is left, 3 is right
depth = generate_depth_map(gt_calib[t_id], 
                                gt_files[t_id], 
                                im_sizes[t_id], 
                                camera_id, 
                                False, 
                                True)

ValueError: cannot reshape array of size 1 into shape (3,3)

In [ ]:
## obtain groundtruths 
gt_depths = []
pred_depths_resized = []
for t_id in range(num_test):
    camera_id = cams[t_id]  # 2 is left, 3 is right
    pred_depths_resized.append(
        cv2.resize(pred_depths[t_id], 
                    (im_sizes[t_id][1], im_sizes[t_id][0]), 
                    interpolation=cv2.INTER_LINEAR))
    depth = generate_depth_map(gt_calib[t_id], 
                                gt_files[t_id], 
                                im_sizes[t_id], 
                                camera_id, 
                                False, 
                                True)
    gt_depths.append(depth.astype(np.float32))

In [6]:
np.save('../sfmLearner_checkpoints/outputs/gt_depths.npy',np.array(gt_depths))
pred_depths = pred_depths_resized

No. of test files not available: 150


ValueError: cannot reshape array of size 1 into shape (3,3)